# Imports

In [1]:
import numpy as np
from numpy import shape
import pandas as pd
import matplotlib.pyplot as plt
#import pydot
from gc import callbacks
from random import shuffle
from tabnanny import verbose
from sklearn.model_selection import train_test_split
import nltk
import nltk.corpus
from nltk.corpus import stopwords, wordnet, words
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2024-07-01 22:06:46.127603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1719871606.766869    2093 cuda_dnn.cc:8453] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1719871606.980514    2093 cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 22:06:48.277701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 22:06:54.660377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[]


In [3]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Data

In [ ]:
df = pd.read_csv('/workspaces/tswift_lyrics/genius_lyrics_collection/tswift_lyrics.csv')
df.head()

,Album,Song Name,Lyrics
0,Taylor Swift,A Perfectly Good Heart,why would you wanna break a perfectly good hea...
1,Taylor Swift,A Place In This World,i dont know what i want so dont ask me cause i...
2,Taylor Swift,Cold as You,you have a way of coming easily to me and when...
3,Taylor Swift,I'm Only Me When I'm With You,friday night beneath the stars in a field behi...
4,Taylor Swift,Invisible,she cant see the way your eyes light up when y...


In [ ]:
# join 'Lyrics' column into single string
lyric_text = ' '.join(df['Lyrics'])

# Train Model

In [ ]:
# training tokenizer on lyric_text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([lyric_text])

# creating vocabulary
total_words = len(tokenizer.word_index) + 1

In [ ]:
# create input sequences
input_sequences = []
for line in df['Lyrics']:
    # convert words into corresponding tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    # print(token_list)
    for i in range(1, len(token_list)):
        # create n-gram sequences from tokens
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print('Total Input Sequences: ', len(input_sequences))

Total Input Sequences:  84261


In [ ]:
# find lyric that has max length
max_sequence_len = max([len(x) for x in input_sequences])

# based on that length, pad the rest of input sequences to have equal length
input_sequences = np.array(pad_sequences(
    input_sequences,
    maxlen=max_sequence_len,
    padding='pre'))

#input_sequences[1]

In [ ]:
# create features and label
# Xs = predictors; ys = output
Xs, labels = input_sequences[:, :-1], input_sequences[:, -1]

# convert class vector to binary class matrix
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(Xs[5])
print(labels[5])
#print(ys[5][14])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [ ]:
# split data
X_train, X_val, y_train, y_val = train_test_split(Xs, labels, test_size=0.25, random_state=42)

In [ ]:
class Perplexity(Callback):
    def on_epoch_end(self, epoch, logs={}):
        cross_entropy = logs.get('loss')
        perplexity = np.exp(cross_entropy)
        print(f' - perplexity: {perplexity}')
        
def train(model):
    # stop training pocess early if model stops improving
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              mode='auto',
                              restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=100,
                        validation_data=(X_val, y_val),
                        callbacks=[earlystop, Perplexity()])
    return history

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel('Epochs')
  plt.ylabel(string)
  plt.show()

: 

# Baseline Model

In [ ]:
baseline_model = Sequential()
# input or embedding layer: transforms input data into dense layers of fixed size (100-D vector)
baseline_model.add(Embedding(total_words, 100)) 
# LSTM layer: 150 LSTM units work to understand the sequence and context of words
baseline_model.add(LSTM(150))
# dropout layer: randomly skips some neurons during training to avoid overfitting
# makes model less sensitive to weights of neurons; set to 0.1 dropout rate
baseline_model.add(Dropout(0.1))
# output or dense layer: prepares model to choose next word
# has as many neurons as there are in vocabulary
baseline_model.add(Dense(total_words, activation='softmax'))
baseline_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

baseline_model.summary()

history = train(baseline_model)

# plot accuracy and loss
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

2024-07-01 20:36:12.517891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-01 20:36:12.530306: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-01 20:36:12.530360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-7820cc): /proc/driver/nvidia/version does not exist
2024-07-01 20:36:12.563051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         487400    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense (Dense)               (None, 4874)              735974    
                                                                 
Total params: 1,373,974
Trainable params: 1,373,974
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# function to generate next word predictions
def gen_predictions(model, seed_text):
  next_words = 150

  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],
                              maxlen=max_sequence_len-1,
                              padding='pre')
    
    #predicted = np.argmax(model.predict(token_list), axis=-1)
    
    # predict next word (as probabilities)
    probabilities = model.predict(token_list, verbose=0)
    # get index of most likely next word
    predicted = np.argmax(probabilities, axis=-1)[0]
    
    output_word = ''
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += ' ' + output_word
  return seed_text

# use baseline model to predict next 150 words with seed text 'I am'
gen_predictions(baseline_model, 'I am')

'I am in the heat of my head in a café i regret you all the time i know why you know that you know that you know that you know that you know that you know that you know that you know is it gets me to you i know places i know why now i know that i hate you now i hate you like i hate you i know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better know you better'

# Model 2

In [ ]:
print('Before splitting:')
print('The shape of the input data is', shape(Xs))
print('The shape of the ouput data is', shape(labels))

print('After splitting:')
print('The shape of the training data is', shape(X_train))
print('The shape of the test data is', shape(X_val))

print((total_words))

Before splitting:
The shape of the input data is (84261, 977)
The shape of the ouput data is (84261, 4874)
After splitting:
The shape of the training data is (63195, 977)
The shape of the test data is (21066, 977)
4874


In [ ]:
# make sure model generates same results everytime it runs
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# batch size
n_batch = len(X_train)

# define and build model2
model2 = Sequential()
# add LSTM as input layer
model2.add(LSTM(256,
                input_shape=(100, X_train.shape[1]),
                             #X_train.shape[1]),
                return_sequences=True))
# add three LSTM hidden layers, 64 nodes in each layer
model2.add(LSTM(256, return_sequences=True))
model2.add(LSTM(256, return_sequences=True))
model2.add(LSTM(256, return_sequences=True))
# flatten data from last hidden layer as it is inputted to output layer
model2.add(Flatten())
# add dense layer, a fully connect neural network layer
model2.add(Dense(y_train.shape[1]))
# use softmax activation since multiclassification model
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

model2.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_90 (LSTM)                  │ (None, 100, 256)       │     1,263,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_91 (LSTM)                  │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_92 (LSTM)                  │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_93 (LSTM)                  │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4874)           │   124,779,274 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 4874)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,618,826 (486.83 MB)

 Trainable params: 127,618,826 (486.83 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# save work at model chkpnts
chkpnt_name = 'Weights-LSTM-Improvement-{epoch:03d}-{loss:.5d}-bigger.keras'
chkpnt = ModelCheckpoint(chkpnt_name,
                         monitor = 'loss',
                         verbose = 1,
                         save_best_only = True,
                         mode = 'min')
callbacks_list = [chkpnt]

In [ ]:
# fit model2
model2_params = {'epochs':10,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'shuffle':True,
                'initial_epoch':0}

model2_history = model2.fit(X_train, y_train,
                           model2_params['epochs'],
                           batch_size = model2_params['batch_size'],
                           callbacks = model2_params['callbacks'],
                           verbose = model2_params['verbose'],
                           validation_split = model2_params['validation_split'],
                           shuffle = model2_params['shuffle'],
                           initial_epoch = model2_params['initial_epoch']
                           )

TypeError: fit() got multiple values for argument 'batch_size'

In [ ]:
# view loss and accuracy history
model2_history.history

# plot loss and accuracy history


In [ ]:
# train model2
history = model2.train(model2)

# plot accuracy and loss
keras.utils.plot_model(model2)
# plot_graphs(history, 'accuracy')
# plot_graphs(history, 'loss')

In [ ]:
# model2.add(Embedding(total_words, 100))
# model2.add(Bidirectional(LSTM(150)))
# #model2.add(LSTM(64))
# model2.add(Dense(total_words, activation='softmax'))
# adam = Adam(learning_rate=0.01)
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# # train model
# history = model.fit(Xs, ys, epochs=50, verbose=1)
# #model.fit(X, y, epochs=500, verbose=1)
# print(model)

# # note: takes about an hour to run!